In [32]:
# =============================
# 🔹 1. Imports & Setup
# =============================
# !pip install yfinance tensorflow scikit-learn pandas matplotlib joblib --quiet

import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Layer, Dense,Dropout, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Add, Multiply, Concatenate, Activation

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


symbol = "BTC-USD"
interval = "5m"   # <-- 5 minutes pour 30 jours (compatible)
period = "30d"    # <-- 30 jours d’historique

df = yf.download(tickers=symbol, interval=interval, period=period)

if df.empty:
    raise ValueError(f"Aucune donnée téléchargée pour {symbol}")

print(f"{df.shape[0]} lignes téléchargées")
df.head()


C:\Users\Miguel\AppData\Local\Temp\ipykernel_23228\1605864730.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=symbol, interval=interval, period=period)
[*********************100%***********************]  1 of 1 completed

8538 lignes téléchargées


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Datetime,,,,,
2025-09-22 00:00:00+00:00,115336.234375,115336.234375,115270.718750,115270.718750,0
2025-09-22 00:05:00+00:00,115380.632812,115430.632812,115374.484375,115374.484375,94775296
2025-09-22 00:10:00+00:00,115384.648438,115410.937500,115356.507812,115356.507812,22927360
2025-09-22 00:15:00+00:00,115285.664062,115376.148438,115285.664062,115341.601562,38602752
2025-09-22 00:20:00+00:00,115235.515625,115268.929688,115232.890625,115268.929688,1695744


In [33]:
# =============================
# 🔹 2. Préparation des données
# =============================

df['logret'] = np.log(df['Close']).diff()
df['vol_30'] = df['logret'].rolling(30).std()
df = df.dropna()

features = ['Close', 'logret', 'vol_30']
data = df[features].astype('float32').values

# Split temporel
train_ratio = 0.8
split_idx = int(len(data) * train_ratio)
train_data, val_data = data[:split_idx], data[split_idx:]

# Normalisation
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
val_scaled   = scaler.transform(val_data)

In [34]:
# =============================
# 🔹 3. Création des séquences (look_back)
# =============================

def make_windows(arr, look_back=128, horizon=3):
    X, y = [], []
    for i in range(len(arr) - look_back - horizon + 1):
        X.append(arr[i:i+look_back])
        y.append(arr[i+look_back:i+look_back+horizon, 0])  # Close futur
    return np.array(X), np.array(y)

look_back, horizon = 128, 3
X_train, y_train = make_windows(train_scaled, look_back, horizon)
X_val, y_val     = make_windows(val_scaled, look_back, horizon)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

Train: (6676, 128, 3), Val: (1572, 128, 3)


In [35]:

class CBAM_Block_1D(Layer):
    def __init__(self, reduction_ratio=8, **kwargs):
        super(CBAM_Block_1D, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.channel = input_shape[-1]
        self.shared_dense_one = Dense(self.channel // self.reduction_ratio, activation='relu')
        self.shared_dense_two = Dense(self.channel)
        self.conv_temporal = Conv1D(filters=1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_pool = GlobalAveragePooling1D()(inputs)
        max_pool = GlobalMaxPooling1D()(inputs)
        avg_out = self.shared_dense_two(self.shared_dense_one(avg_pool))
        max_out = self.shared_dense_two(self.shared_dense_one(max_pool))
        channel_attention = Activation('sigmoid')(Add()([avg_out, max_out]))
        channel_attention = tf.expand_dims(channel_attention, axis=1)
        x = Multiply()([inputs, channel_attention])
        avg_pool_t = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool_t = tf.reduce_max(x, axis=-1, keepdims=True)
        concat = Concatenate(axis=-1)([avg_pool_t, max_pool_t])
        temporal_attention = self.conv_temporal(concat)
        x = Multiply()([x, temporal_attention])
        return x

In [36]:
# =============================
# 🔹 4. Construction du modèle LSTM
# =============================
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(look_back, len(features))),
    Dropout(0.2),
    CBAM_Block_1D(),
    LSTM(64),
    Dropout(0.2),
    Dense(horizon)
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
              loss='mse',
              metrics=['mae'])
model.summary()

c:\Users\Miguel\anaconda3\envs\tfcpu\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 128, 128)       │        67,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cbam__block_1d_2                │ (None, 128, 128)       │         4,255 │
│ (CBAM_Block_1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,442 (474.38 KB)

 Trainable params: 121,442 (474.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    ModelCheckpoint('best_cbam_lstm.keras', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,         # tu peux mettre 20 si tu veux tester d’abord
    batch_size=32,     # petit batch => meilleur résultat
    shuffle=False,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/20


In [ ]:
y_pred = model.predict(X_val)
plt.figure(figsize=(10, 5))
plt.plot(y_val[:, 0], label='True t+1')
plt.plot(y_pred[:, 0], label='Pred t+1')
plt.title('Prédiction CBAM LSTM (t+1)')
plt.legend()
plt.show()


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step


ValueError: y_true and y_pred have different number of output (1!=3)

In [ ]:
# =============================
# 🔹 7. Sauvegarde du modèle et du scaler
# =============================
# model.save("model.h5")
# joblib.dump({"scaler": scaler, "look_back": look_back}, "preproc.pkl")

print("✅ Modèle et scaler sauvegardés avec succès !")


✅ Modèle et scaler sauvegardés avec succès !
